In [1]:
import os
os.getcwd()

'C:\\Users\\Jayanth\\Python_DataAnalysis\\PySpark-lessons'

In [2]:
import pyspark
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('WindowFunctions').getOrCreate()

- Window.partitionBy: This can be used to categorize data and perform analysis within sub categories. Comma separated multiple columns can be added in window partition. Example: In order to calculate sum of salary for each Department along with individual detail, partition by dept number is required. 

In [4]:
df = spark.read.parquet("file:///C:\\Users\\Jayanth\\Python_DataAnalysis\\PySpark-lessons\\datasets\\emp_data.parquet")
df.show()

+-------+--------+-------+----------+-------+-----+
| emp_no|emp_name| salary|manager_id|dept_no| comm|
+-------+--------+-------+----------+-------+-----+
|1000245| PRADEEP|5000.00|      null|    100| 0.00|
|1000258|   BLAKE|2850.00|   1000245|    300|50.00|
|1000262|   CLARK|2450.00|   1000245|    100|50.00|
|1000276|   JONES|2975.00|   1000245|    200|75.00|
|1000288|   SCOTT|3000.00|   1000276|    200| 0.00|
|1000292|    FORD|3000.00|   1000276|    200| 0.00|
|1000294|   SMITH| 800.00|   1000292|    200| 0.00|
|1000299|   ALLEN|1600.00|   1000258|    300| 0.00|
|1000310|    WARD|1250.00|   1000258|    300|50.00|
|1000312|  MARTIN|1250.00|   1000258|    300|50.00|
|1000315|  TURNER|1500.00|   1000258|    300| 0.00|
|1000326|   ADAMS|1100.00|   1000288|    200| 0.00|
|1000336|   JAMES| 950.00|   1000258|    300|50.00|
|1000346|  MILLER|1300.00|   1000262|    100| 0.00|
|1000347|   DAVID|1400.00|   1000245|    500| 0.00|
+-------+--------+-------+----------+-------+-----+



In [5]:
df.printSchema()
df.show(truncate=False)

root
 |-- emp_no: integer (nullable = true)
 |-- emp_name: string (nullable = true)
 |-- salary: decimal(18,2) (nullable = true)
 |-- manager_id: integer (nullable = true)
 |-- dept_no: integer (nullable = true)
 |-- comm: decimal(5,2) (nullable = true)

+-------+--------+-------+----------+-------+-----+
|emp_no |emp_name|salary |manager_id|dept_no|comm |
+-------+--------+-------+----------+-------+-----+
|1000245|PRADEEP |5000.00|null      |100    |0.00 |
|1000258|BLAKE   |2850.00|1000245   |300    |50.00|
|1000262|CLARK   |2450.00|1000245   |100    |50.00|
|1000276|JONES   |2975.00|1000245   |200    |75.00|
|1000288|SCOTT   |3000.00|1000276   |200    |0.00 |
|1000292|FORD    |3000.00|1000276   |200    |0.00 |
|1000294|SMITH   |800.00 |1000292   |200    |0.00 |
|1000299|ALLEN   |1600.00|1000258   |300    |0.00 |
|1000310|WARD    |1250.00|1000258   |300    |50.00|
|1000312|MARTIN  |1250.00|1000258   |300    |50.00|
|1000315|TURNER  |1500.00|1000258   |300    |0.00 |
|1000326|ADAMS   

- Window.orderBy:can be used to specify order in which rows should be processed. 

- count function(): count function can be used to count number of records for each group. 

In [6]:
from pyspark.sql.window import  Window
from pyspark.sql.functions import count, col

empdf = df.select("emp_name", "dept_no", "salary", count("*").over(Window.partitionBy("dept_no")).alias("Total_Emps"))
empdf.show()

+--------+-------+-------+----------+
|emp_name|dept_no| salary|Total_Emps|
+--------+-------+-------+----------+
| PRADEEP|    100|5000.00|         3|
|   CLARK|    100|2450.00|         3|
|  MILLER|    100|1300.00|         3|
|   JONES|    200|2975.00|         5|
|   SCOTT|    200|3000.00|         5|
|    FORD|    200|3000.00|         5|
|   SMITH|    200| 800.00|         5|
|   ADAMS|    200|1100.00|         5|
|   BLAKE|    300|2850.00|         6|
|   ALLEN|    300|1600.00|         6|
|    WARD|    300|1250.00|         6|
|  MARTIN|    300|1250.00|         6|
|  TURNER|    300|1500.00|         6|
|   JAMES|    300| 950.00|         6|
|   DAVID|    500|1400.00|         1|
+--------+-------+-------+----------+



**Window.rowsBetween:** Can be used when user want to peek into other rows data to compute current row value. 
   - rowsBetween( Window.unboundedPreceding|Window.currentRow|Window.currentRow+-n (start window) , Window.currentRow+-m |Window.currentRow| Window.unboundedFollowing (end window))
   - rowsBetween attribute require user to specify start window and end window of row. 
   - Window.unboundedPreceding, Window.unboundedFollowing and Window.currentRow values are available to peek into previous rows, following rows and current row respectively. 
   - User can add or subtract number(e.g 1,2,3) from Window.currentRow to check that many number of previous / following rows or specify UNBOUNDED attributes to check all the previous / following rows. 
    

In [7]:
empdf = df.select("emp_name", "dept_no", "salary", count("*").over(Window.partitionBy("dept_no").rowsBetween(Window.currentRow+1, Window.unboundedFollowing)).alias("Remaining_cnt"))
empdf.show()

+--------+-------+-------+-------------+
|emp_name|dept_no| salary|Remaining_cnt|
+--------+-------+-------+-------------+
| PRADEEP|    100|5000.00|            2|
|   CLARK|    100|2450.00|            1|
|  MILLER|    100|1300.00|            0|
|   JONES|    200|2975.00|            4|
|   SCOTT|    200|3000.00|            3|
|    FORD|    200|3000.00|            2|
|   SMITH|    200| 800.00|            1|
|   ADAMS|    200|1100.00|            0|
|   BLAKE|    300|2850.00|            5|
|   ALLEN|    300|1600.00|            4|
|    WARD|    300|1250.00|            3|
|  MARTIN|    300|1250.00|            2|
|  TURNER|    300|1500.00|            1|
|   JAMES|    300| 950.00|            0|
|   DAVID|    500|1400.00|            0|
+--------+-------+-------+-------------+



**sum function():** sum function can be used to calculate sum of each column passed to this function for each group. This function can be applied to only numeric columns.

- Cumulative sum of salaries grouped over department number(dept_no). There will be problem when order by values are same within a partition

In [8]:
from pyspark.sql.window import  Window
from pyspark.sql.functions import sum, col

empdf = df.select("emp_name", "dept_no", "salary", sum("salary").over(Window.partitionBy("dept_no").orderBy( col("salary").desc())).alias("cummulative_sum"))
empdf.show(5)

+--------+-------+-------+---------------+
|emp_name|dept_no| salary|cummulative_sum|
+--------+-------+-------+---------------+
| PRADEEP|    100|5000.00|        5000.00|
|   CLARK|    100|2450.00|        7450.00|
|  MILLER|    100|1300.00|        8750.00|
|   SCOTT|    200|3000.00|        6000.00|
|    FORD|    200|3000.00|        6000.00|
+--------+-------+-------+---------------+
only showing top 5 rows



- True cumulative sum of salaries within each department, use multiple order by columns to help spark distinguish rows. 


In [25]:
from pyspark.sql.window import  Window
from pyspark.sql.functions import sum, col

empdf = df.select("emp_name", "dept_no", "salary", sum("salary").over(Window.partitionBy("dept_no").orderBy( col("salary").desc(), col("emp_name").desc())).alias("true_cummulative_sum"))
empdf.show(10)

+--------+-------+-------+--------------------+
|emp_name|dept_no| salary|true_cummulative_sum|
+--------+-------+-------+--------------------+
| PRADEEP|    100|5000.00|             5000.00|
|   CLARK|    100|2450.00|             7450.00|
|  MILLER|    100|1300.00|             8750.00|
|   SCOTT|    200|3000.00|             3000.00|
|    FORD|    200|3000.00|             6000.00|
|   JONES|    200|2975.00|             8975.00|
|   ADAMS|    200|1100.00|            10075.00|
|   SMITH|    200| 800.00|            10875.00|
|   BLAKE|    300|2850.00|             2850.00|
|   ALLEN|    300|1600.00|             4450.00|
+--------+-------+-------+--------------------+
only showing top 10 rows



- Cumulative sum of salaries without any partition.

In [26]:
from pyspark.sql.window import  Window
from pyspark.sql.functions import sum, col

empdf = df.select("emp_name", "dept_no", "salary", sum("salary").over(Window.orderBy( col("salary").desc(), col("emp_name").desc())).alias("true_cummulative_sum"))

empdf.show()

+--------+-------+-------+--------------------+
|emp_name|dept_no| salary|true_cummulative_sum|
+--------+-------+-------+--------------------+
| PRADEEP|    100|5000.00|             5000.00|
|   SCOTT|    200|3000.00|             8000.00|
|    FORD|    200|3000.00|            11000.00|
|   JONES|    200|2975.00|            13975.00|
|   BLAKE|    300|2850.00|            16825.00|
|   CLARK|    100|2450.00|            19275.00|
|   ALLEN|    300|1600.00|            20875.00|
|  TURNER|    300|1500.00|            22375.00|
|   DAVID|    500|1400.00|            23775.00|
|  MILLER|    100|1300.00|            25075.00|
|    WARD|    300|1250.00|            26325.00|
|  MARTIN|    300|1250.00|            27575.00|
|   ADAMS|    200|1100.00|            28675.00|
|   JAMES|    300| 950.00|            29625.00|
|   SMITH|    200| 800.00|            30425.00|
+--------+-------+-------+--------------------+



- Sum of salaries within each department 

In [27]:
from pyspark.sql.window import  Window
from pyspark.sql.functions import sum, col

empdf = df.select("emp_name", "dept_no", "salary", sum("salary").over(Window.partitionBy("dept_no").orderBy( col("salary").desc(), col("emp_name").desc()).rowsBetween(Window.unboundedPreceding, Window.unboundedFollowing)).alias("true_cummulative_sum"))

empdf.show()

+--------+-------+-------+--------------------+
|emp_name|dept_no| salary|true_cummulative_sum|
+--------+-------+-------+--------------------+
| PRADEEP|    100|5000.00|             8750.00|
|   CLARK|    100|2450.00|             8750.00|
|  MILLER|    100|1300.00|             8750.00|
|   SCOTT|    200|3000.00|            10875.00|
|    FORD|    200|3000.00|            10875.00|
|   JONES|    200|2975.00|            10875.00|
|   ADAMS|    200|1100.00|            10875.00|
|   SMITH|    200| 800.00|            10875.00|
|   BLAKE|    300|2850.00|             9400.00|
|   ALLEN|    300|1600.00|             9400.00|
|  TURNER|    300|1500.00|             9400.00|
|    WARD|    300|1250.00|             9400.00|
|  MARTIN|    300|1250.00|             9400.00|
|   JAMES|    300| 950.00|             9400.00|
|   DAVID|    500|1400.00|             1400.00|
+--------+-------+-------+--------------------+



**avg function():** sum function can be used to calculate sum of each column passed to this function for each group. This function can be applied to only numeric columns.
- Average salary within each department 


In [28]:
from pyspark.sql.window import  Window
from pyspark.sql.functions import avg, col

empdf = df.select("emp_name", "dept_no", "salary", avg("salary").over(Window.partitionBy("dept_no")).alias("Average_dept_salary"))
empdf.show(10)

+--------+-------+-------+-------------------+
|emp_name|dept_no| salary|Average_dept_salary|
+--------+-------+-------+-------------------+
| PRADEEP|    100|5000.00|        2916.666667|
|   CLARK|    100|2450.00|        2916.666667|
|  MILLER|    100|1300.00|        2916.666667|
|   JONES|    200|2975.00|        2175.000000|
|   SCOTT|    200|3000.00|        2175.000000|
|    FORD|    200|3000.00|        2175.000000|
|   SMITH|    200| 800.00|        2175.000000|
|   ADAMS|    200|1100.00|        2175.000000|
|   BLAKE|    300|2850.00|        1566.666667|
|   ALLEN|    300|1600.00|        1566.666667|
+--------+-------+-------+-------------------+
only showing top 10 rows



- Cumulative Average salary within each department 

In [29]:
from pyspark.sql.window import  Window
from pyspark.sql.functions import avg, col

empdf = df.select("emp_name", "dept_no", "salary", avg("salary").over(Window.partitionBy("dept_no").orderBy("emp_name").rowsBetween(Window.unboundedPreceding, Window.currentRow)).alias("Cum_avg_Dept_salary"))


empdf.show()

+--------+-------+-------+-------------------+
|emp_name|dept_no| salary|Cum_avg_Dept_salary|
+--------+-------+-------+-------------------+
|   CLARK|    100|2450.00|        2450.000000|
|  MILLER|    100|1300.00|        1875.000000|
| PRADEEP|    100|5000.00|        2916.666667|
|   ADAMS|    200|1100.00|        1100.000000|
|    FORD|    200|3000.00|        2050.000000|
|   JONES|    200|2975.00|        2358.333333|
|   SCOTT|    200|3000.00|        2518.750000|
|   SMITH|    200| 800.00|        2175.000000|
|   ALLEN|    300|1600.00|        1600.000000|
|   BLAKE|    300|2850.00|        2225.000000|
|   JAMES|    300| 950.00|        1800.000000|
|  MARTIN|    300|1250.00|        1662.500000|
|  TURNER|    300|1500.00|        1630.000000|
|    WARD|    300|1250.00|        1566.666667|
|   DAVID|    500|1400.00|        1400.000000|
+--------+-------+-------+-------------------+



- Specifying the data as follows

In [32]:
windowSpecAgg  = Window.partitionBy("dept_no")
from pyspark.sql.functions import col,avg,sum,min,max,row_number 
df.withColumn("row",row_number().over(windowSpec)) \
  .withColumn("avg", avg(col("salary")).over(windowSpecAgg)) \
  .withColumn("sum", sum(col("salary")).over(windowSpecAgg)) \
  .withColumn("min", min(col("salary")).over(windowSpecAgg)) \
  .withColumn("max", max(col("salary")).over(windowSpecAgg)) \
  .where(col("row")==1).select("dept_no","avg","sum","min","max") \
  .show()

+-------+-----------+--------+-------+-------+
|dept_no|        avg|     sum|    min|    max|
+-------+-----------+--------+-------+-------+
|    100|2916.666667| 8750.00|1300.00|5000.00|
|    200|2175.000000|10875.00| 800.00|3000.00|
|    300|1566.666667| 9400.00| 950.00|2850.00|
|    500|1400.000000| 1400.00|1400.00|1400.00|
+-------+-----------+--------+-------+-------+



In [7]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
windowSpec  = Window.partitionBy("dept_no").orderBy("salary")

In [8]:
df.withColumn("row_number",row_number().over(windowSpec)) \
    .show(truncate=False)

+-------+--------+-------+----------+-------+-----+----------+
|emp_no |emp_name|salary |manager_id|dept_no|comm |row_number|
+-------+--------+-------+----------+-------+-----+----------+
|1000346|MILLER  |1300.00|1000262   |100    |0.00 |1         |
|1000262|CLARK   |2450.00|1000245   |100    |50.00|2         |
|1000245|PRADEEP |5000.00|null      |100    |0.00 |3         |
|1000294|SMITH   |800.00 |1000292   |200    |0.00 |1         |
|1000326|ADAMS   |1100.00|1000288   |200    |0.00 |2         |
|1000276|JONES   |2975.00|1000245   |200    |75.00|3         |
|1000288|SCOTT   |3000.00|1000276   |200    |0.00 |4         |
|1000292|FORD    |3000.00|1000276   |200    |0.00 |5         |
|1000336|JAMES   |950.00 |1000258   |300    |50.00|1         |
|1000310|WARD    |1250.00|1000258   |300    |50.00|2         |
|1000312|MARTIN  |1250.00|1000258   |300    |50.00|3         |
|1000315|TURNER  |1500.00|1000258   |300    |0.00 |4         |
|1000299|ALLEN   |1600.00|1000258   |300    |0.00 |5   

- rank of the salary for each department

In [9]:
from pyspark.sql.functions import rank
df.withColumn("rank",rank().over(windowSpec)) \
    .show()

+-------+--------+-------+----------+-------+-----+----+
| emp_no|emp_name| salary|manager_id|dept_no| comm|rank|
+-------+--------+-------+----------+-------+-----+----+
|1000346|  MILLER|1300.00|   1000262|    100| 0.00|   1|
|1000262|   CLARK|2450.00|   1000245|    100|50.00|   2|
|1000245| PRADEEP|5000.00|      null|    100| 0.00|   3|
|1000294|   SMITH| 800.00|   1000292|    200| 0.00|   1|
|1000326|   ADAMS|1100.00|   1000288|    200| 0.00|   2|
|1000276|   JONES|2975.00|   1000245|    200|75.00|   3|
|1000288|   SCOTT|3000.00|   1000276|    200| 0.00|   4|
|1000292|    FORD|3000.00|   1000276|    200| 0.00|   4|
|1000336|   JAMES| 950.00|   1000258|    300|50.00|   1|
|1000310|    WARD|1250.00|   1000258|    300|50.00|   2|
|1000312|  MARTIN|1250.00|   1000258|    300|50.00|   2|
|1000315|  TURNER|1500.00|   1000258|    300| 0.00|   4|
|1000299|   ALLEN|1600.00|   1000258|    300| 0.00|   5|
|1000258|   BLAKE|2850.00|   1000245|    300|50.00|   6|
|1000347|   DAVID|1400.00|   10

- Unique row number against employees by salary. 

In [12]:
from pyspark.sql.window import  Window
from pyspark.sql.functions import row_number, col

empdf_update = df.select("emp_name", "dept_no", "salary", row_number().over(Window.orderBy("salary")).alias("row_number_salary"))
empdf_update.show()

+--------+-------+-------+-----------------+
|emp_name|dept_no| salary|row_number_salary|
+--------+-------+-------+-----------------+
|   SMITH|    200| 800.00|                1|
|   JAMES|    300| 950.00|                2|
|   ADAMS|    200|1100.00|                3|
|    WARD|    300|1250.00|                4|
|  MARTIN|    300|1250.00|                5|
|  MILLER|    100|1300.00|                6|
|   DAVID|    500|1400.00|                7|
|  TURNER|    300|1500.00|                8|
|   ALLEN|    300|1600.00|                9|
|   CLARK|    100|2450.00|               10|
|   BLAKE|    300|2850.00|               11|
|   JONES|    200|2975.00|               12|
|   SCOTT|    200|3000.00|               13|
|    FORD|    200|3000.00|               14|
| PRADEEP|    100|5000.00|               15|
+--------+-------+-------+-----------------+



- rank function(): This function is used to find rank within partitions or sub-partitions. Rank function will miss next number(rank) if there are 2 records with same value.
- Row number against employees by salary within department. 

In [14]:
from pyspark.sql.window import  Window
from pyspark.sql.functions import row_number, col

empdf_update = df.select("emp_name", "dept_no", "salary", row_number().over(Window.partitionBy("dept_no").orderBy("salary")).alias("row_number_Dept_salary"))
empdf_update.show()

+--------+-------+-------+----------------------+
|emp_name|dept_no| salary|row_number_Dept_salary|
+--------+-------+-------+----------------------+
|  MILLER|    100|1300.00|                     1|
|   CLARK|    100|2450.00|                     2|
| PRADEEP|    100|5000.00|                     3|
|   SMITH|    200| 800.00|                     1|
|   ADAMS|    200|1100.00|                     2|
|   JONES|    200|2975.00|                     3|
|   SCOTT|    200|3000.00|                     4|
|    FORD|    200|3000.00|                     5|
|   JAMES|    300| 950.00|                     1|
|    WARD|    300|1250.00|                     2|
|  MARTIN|    300|1250.00|                     3|
|  TURNER|    300|1500.00|                     4|
|   ALLEN|    300|1600.00|                     5|
|   BLAKE|    300|2850.00|                     6|
|   DAVID|    500|1400.00|                     1|
+--------+-------+-------+----------------------+



-  Rank of employees by salary within department. 

In [18]:
from pyspark.sql.window import  Window
from pyspark.sql.functions import rank, col

empdf_update = df.select("emp_name", "dept_no", "salary", rank().over(Window.partitionBy("dept_no").orderBy( col("salary").desc())).alias("Rank_by_salary"))
empdf_update.show(truncate=False)

+--------+-------+-------+--------------+
|emp_name|dept_no|salary |Rank_by_salary|
+--------+-------+-------+--------------+
|PRADEEP |100    |5000.00|1             |
|CLARK   |100    |2450.00|2             |
|MILLER  |100    |1300.00|3             |
|SCOTT   |200    |3000.00|1             |
|FORD    |200    |3000.00|1             |
|JONES   |200    |2975.00|3             |
|ADAMS   |200    |1100.00|4             |
|SMITH   |200    |800.00 |5             |
|BLAKE   |300    |2850.00|1             |
|ALLEN   |300    |1600.00|2             |
|TURNER  |300    |1500.00|3             |
|WARD    |300    |1250.00|4             |
|MARTIN  |300    |1250.00|4             |
|JAMES   |300    |950.00 |6             |
|DAVID   |500    |1400.00|1             |
+--------+-------+-------+--------------+



#### Creating a table name with emp_data_csv

In [12]:
temp_table_name = "emp_data_csv"
df.createOrReplaceTempView(temp_table_name)

In [13]:
permanent_table_name = "emp_data_table"
df.write.format("parquet").saveAsTable(permanent_table_name)

In [14]:
spark.sql("select * from emp_data_table").show()

+-------+--------+-------+----------+-------+-----+
| emp_no|emp_name| salary|manager_id|dept_no| comm|
+-------+--------+-------+----------+-------+-----+
|1000245| PRADEEP|5000.00|      null|    100| 0.00|
|1000258|   BLAKE|2850.00|   1000245|    300|50.00|
|1000262|   CLARK|2450.00|   1000245|    100|50.00|
|1000276|   JONES|2975.00|   1000245|    200|75.00|
|1000288|   SCOTT|3000.00|   1000276|    200| 0.00|
|1000292|    FORD|3000.00|   1000276|    200| 0.00|
|1000294|   SMITH| 800.00|   1000292|    200| 0.00|
|1000299|   ALLEN|1600.00|   1000258|    300| 0.00|
|1000310|    WARD|1250.00|   1000258|    300|50.00|
|1000312|  MARTIN|1250.00|   1000258|    300|50.00|
|1000315|  TURNER|1500.00|   1000258|    300| 0.00|
|1000326|   ADAMS|1100.00|   1000288|    200| 0.00|
|1000336|   JAMES| 950.00|   1000258|    300|50.00|
|1000346|  MILLER|1300.00|   1000262|    100| 0.00|
|1000347|   DAVID|1400.00|   1000245|    500| 0.00|
+-------+--------+-------+----------+-------+-----+



In [17]:
#Pyspark
#reading data
df = spark.sql("select * from emp_data_table")


# Aggregate Window Functions
from pyspark.sql.functions import col,avg,sum,min,max,count
from pyspark.sql import Window

df = df.withColumn("avg_salary", avg(col("salary")).over(Window.partitionBy("dept_no").orderBy("salary")))
print(df.show())
df = df.withColumn("max_salary", max(col("salary")).over(Window.partitionBy("dept_no")))
print(df.show())
df = df.withColumn("min_salary", min(col("salary")).over(Window.partitionBy("dept_no")))
print(df.show())
df = df.withColumn("sum_salary", sum(col("salary")).over(Window.partitionBy("dept_no").orderBy("salary")))
print(df.show())
df = df.withColumn("count_salary_units", count(col("salary")).over(Window.partitionBy("dept_no")))
print(df.show())

+-------+--------+-------+----------+-------+-----+-----------+
| emp_no|emp_name| salary|manager_id|dept_no| comm| avg_salary|
+-------+--------+-------+----------+-------+-----+-----------+
|1000346|  MILLER|1300.00|   1000262|    100| 0.00|1300.000000|
|1000262|   CLARK|2450.00|   1000245|    100|50.00|1875.000000|
|1000245| PRADEEP|5000.00|      null|    100| 0.00|2916.666667|
|1000294|   SMITH| 800.00|   1000292|    200| 0.00| 800.000000|
|1000326|   ADAMS|1100.00|   1000288|    200| 0.00| 950.000000|
|1000276|   JONES|2975.00|   1000245|    200|75.00|1625.000000|
|1000288|   SCOTT|3000.00|   1000276|    200| 0.00|2175.000000|
|1000292|    FORD|3000.00|   1000276|    200| 0.00|2175.000000|
|1000336|   JAMES| 950.00|   1000258|    300|50.00| 950.000000|
|1000310|    WARD|1250.00|   1000258|    300|50.00|1150.000000|
|1000312|  MARTIN|1250.00|   1000258|    300|50.00|1150.000000|
|1000315|  TURNER|1500.00|   1000258|    300| 0.00|1237.500000|
|1000299|   ALLEN|1600.00|   1000258|   

#### finding the row number, rank, dense rank, percent rank and ntile

In [9]:
from pyspark.sql.functions import row_number, rank, dense_rank, ntile, percent_rank
from pyspark.sql import Window

df = df.withColumn("row_number", row_number().over(Window.partitionBy("dept_no").orderBy("salary")))
df = df.withColumn("rank", rank().over(Window.partitionBy("dept_no").orderBy("salary")))
df = df.withColumn("dense_rank", dense_rank().over(Window.partitionBy("dept_no").orderBy("salary")))
df = df.withColumn("percent_rank", percent_rank().over(Window.partitionBy("dept_no").orderBy("salary")))
df = df.withColumn("ntile", ntile(10).over(Window.partitionBy("dept_no").orderBy("salary")))


In [14]:
df.select("emp_no","emp_name","dept_no","salary","row_number","rank","dense_rank","percent_rank","ntile").distinct().filter(df.manager_id=="1000258").show()

+-------+--------+-------+-------+----------+----+----------+------------+-----+
| emp_no|emp_name|dept_no| salary|row_number|rank|dense_rank|percent_rank|ntile|
+-------+--------+-------+-------+----------+----+----------+------------+-----+
|1000336|   JAMES|    300| 950.00|         1|   1|         1|         0.0|    1|
|1000310|    WARD|    300|1250.00|         2|   2|         2|         0.2|    2|
|1000312|  MARTIN|    300|1250.00|         3|   2|         2|         0.2|    3|
|1000315|  TURNER|    300|1500.00|         4|   4|         3|         0.6|    4|
|1000299|   ALLEN|    300|1600.00|         5|   5|         4|         0.8|    5|
+-------+--------+-------+-------+----------+----+----------+------------+-----+



#### LEAD, LAG, FIRST_VALUE, LAST_VALUE, NTH_VALUE

In [1]:
from pyspark.sql.functions import cume_dist

In [6]:
?cume_dist

In [15]:

# Analytical Window Functions
from pyspark.sql.functions import cume_dist, lag, lead
from pyspark.sql import Window

df = df.withColumn("cume_dist", cume_dist().over(Window.partitionBy("dept_no").orderBy("salary")))
df = df.withColumn("lag", lag("salary").over(Window.partitionBy("dept_no").orderBy("salary")))
df = df.withColumn("lead", lead("salary").over(Window.partitionBy("dept_no").orderBy("salary")))

#df.display()
#df.distinct().display()
df.select("dept_no","salary","cume_dist","lag","lead").distinct().filter(df.manager_id=="1000258").show()

+-------+-------+-------------------+-------+-------+
|dept_no| salary|          cume_dist|    lag|   lead|
+-------+-------+-------------------+-------+-------+
|    300|1250.00|                0.5| 950.00|1250.00|
|    300|1250.00|                0.5|1250.00|1500.00|
|    300|1500.00| 0.6666666666666666|1250.00|1600.00|
|    300|1600.00| 0.8333333333333334|1500.00|2850.00|
|    300| 950.00|0.16666666666666666|   null|1250.00|
+-------+-------+-------------------+-------+-------+

